# My Code Start From Here
### First part




In [0]:
import pandas as pd
import numpy as np
import spacy
from nltk import ngrams, pos_tag, word_tokenize
from collections import Counter
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
def get_corpus_and_titles():
  df = pd.read_csv('https://bit.ly/nlp-reuters')
  print("Fetching corpus and titles")
  corpus = df.content.to_list()
  titles = df.title.to_list()
  return corpus, titles

In [0]:
corpus = get_corpus_and_titles()[0]
titles = get_corpus_and_titles()[1]

Fetching corpus and titles
Fetching corpus and titles


In [0]:
corpus[0]

'Hey Siri, where can I get my cracked iPhone screen fixed? Apple Inc customers will soon have more choices as the company looks to reduce long wait times for iPhone repairs at its retail stores. By the end of 2017, Apple will to put its proprietary machines for mending cracked iPhone glass in about 400 authorized   repair centers in 25 countries, company executives told Reuters.  Among the first recipients is   Best Buy, which has long sold and serviced Apple products. The electronics retailer already has one of the   machines at a   store and one coming soon to an outlet in Sunnyvale, California.  Fixing cracked screens may seem like small potatoes, but it’s a    global business. The move is also a major shift for Apple. The company had previously restricted use of its   Horizon Machine to its nearly 500 retail stores and   repair centers; and it has guarded its design closely.  The change also comes as eight U. S. states have launched ”right to repair” bills aimed at prying open the 

In [0]:
# No need to remove punctuation, no need to remove stop words
def get_2grams(corpus):
  vocabulary = Counter()
  # using NLTK word_tokenize
  for cont in corpus:
    tokens = word_tokenize(cont)
    for grams in ngrams(pos_tag(tokens), 2):
      # where both tokens are PROPER NOUNS
      if grams[0][1] == 'NNP' and grams[1][1] == 'NNP':
        # needs a [tokens] to maintain tuple, or it'll be split
        vocabulary.update([grams])
  return vocabulary


#### 5 most frequent 2-grams (where both tokens are PROPER NOUNS)

In [0]:
get_2grams(corpus).most_common(5)

[((('U.', 'NNP'), ('S.', 'NNP')), 17600),
 ((('Donald', 'NNP'), ('Trump', 'NNP')), 3211),
 ((('Trump', 'NNP'), ('’', 'NNP')), 2789),
 ((('New', 'NNP'), ('York', 'NNP')), 2443),
 ((('Islamic', 'NNP'), ('State', 'NNP')), 1926)]

# Second part

In [0]:
# from the Buzzfeed News dataset (content), available at bit.ly/nlp-buzzfeed
def get_buzzfeed():
  df = pd.read_csv('https://bit.ly/nlp-buzzfeed')
  print("Fetching corpus and titles")
  corpus = df.content.to_list()
  titles = df.title.to_list()
  return corpus, titles

In [0]:
buzzfeed_corpus = get_buzzfeed()[0]

Fetching corpus and titles


In [0]:
def tfidf_vectors(corpus, vocab):

  # return the TF-IDF vector for a single document
  def tfidf_vec(doc):
    doc_freqs = Counter(doc)
    return [doc_freqs[token] * math.log(N / df[token]) for token, _ in vocab]

  # first, compute the document frequency; df[token] is the number of documents containing token
  df = Counter()
  token_set = set([token for token, _ in vocab])
  for document in corpus:
    df.update(list(set(document) & token_set))

  N = len(corpus)
  # use df to compute the df-idf vector for each document

  return [tfidf_vec(doc) for doc in corpus]

### buzzfeed_corpus has NaN (float type)!

In [0]:
type(buzzfeed_corpus[60])

float

In [0]:
#@title Default title text
from tqdm.notebook import tqdm 

import spacy
# use the SpaCy POS tagger and tokenizer (en_core_web_sm model)
nlp = spacy.load("en_core_web_sm")
# tokenizer?
docs = []
tokens = []
count = 0
for item in tqdm(buzzfeed_corpus):
  if isinstance(item, float):
    docs.append('nan')
  else:
    doc = nlp(item)
    for token in doc:
      # print(token.text, token.pos_, token.lemma_, token.is_alpha, token.is_stop) # stopwords or not,alphabet
      
      # remove stopwords (can use token.is_stop)
      # remove punctuation (maybe should do this)
      # remove space (maybe should do this)
      if token.is_stop is False and token.is_punct is False and token.is_space is False:
        # use each token’s combined lemma + POS (e.g., “give_VERB”) as the term for input to TF-IDF
        tokens.append(token.lemma_ + "_" + token.pos_)
    docs.append(tokens)
    tokens = []

In [0]:
# use the 512 most frequent terms as features 
freq = Counter()
for doc in tqdm(docs):
  for token in doc:
    freq.update([token])

In [0]:
tf = freq.most_common(512)

In [0]:
import math
vectors = tfidf_vectors(docs, tf)

In [0]:
def cosine_similarity(vec_a, vec_b):
  assert len(vec_a) == len(vec_b)
  a_b = sum(i[0] * i[1] for i in zip(vec_a, vec_b))
  a_2 = sum([i*i for i in vec_a])
  b_2 = sum([i*i for i in vec_b])
  if a_2 == 0 or b_2 == 0:
    return float('nan')
  return a_b/(math.sqrt(a_2) * math.sqrt(b_2))
  
def k_similar(seed_id, vectors, k=5):
  similarities = [cosine_similarity(vectors[seed_id], vectors[id]) for id in range(len(vectors))]
  top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i])[-k:] # https://stackoverflow.com/questions/13070461/get-indices-of-the-top-n-values-of-a-list
  return [(id, similarities[id]) for id in reversed(top_indices)]

In [0]:
while True:
  # 5 most similar articles to seed_id = <student_ID> % 1000
  seed_id = 853426 % 1000
  if seed_id >= len(corpus):
    print("No such article.")
    break
  print("* {}".format(titles[seed_id]))
  for id, similarity in k_similar(seed_id, vectors):
    if seed_id == id: # no need to recommend the original article
      continue
    print(" {:4} > {} ({:.4f})".format(id, titles[id], similarity))
  print()
  break

* Verizon closing call centers in five U.S. states, 3,200 jobs affected
  507 > Corporate Japan unimpressed with BOJ’s latest attempt to spur growth: Reuters poll (0.4976)
  692 > Judge blocks Obama rule extending overtime pay to 4.2 million U.S. workers (0.3914)
  682 > Second wind for stocks as earnings turn positive (0.3786)
  727 > November U.S. auto sales up 3.7 percent on back of steep discounts (0.3736)
  764 > Fed on course to raise interest rates at an upcoming meeting: Yellen (0.3708)

